# Gerekli Kütüphaneler

In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn import metrics

seed = 42
np.random.seed(seed)
random.seed(seed)

# Veri Seti

In [2]:
X, y = make_blobs(n_samples=100, centers=3, n_features=2, random_state=seed)

# Mean Shift

In [19]:
class MeanShift:
    def __init__(self, bandwidth=None, max_iter=100, tol=1e-3):
        self.bandwidth = bandwidth
        self.max_iter = max_iter
        self.tol = tol
        self.cluster_centers_ = None
        self.labels_ = None

    def fit(self, X):
        if self.bandwidth is None:
            # Scott kurali
            self.bandwidth = np.mean(np.std(X, axis=0)) * 1.06 * X.shape[0] ** (-1 / 5.)

        # Veri noktalari kopyalanir
        points = np.copy(X)
        for _ in range(self.max_iter):
            new_points = []
            for point in points:
                # Bandwidth icinde olan noktalar bulunur
                in_bandwidth = []
                for x in X:
                    dist = np.sqrt(np.sum((point - x) ** 2))
                    if dist < self.bandwidth:
                        in_bandwidth.append(x)
    
                new_point = np.mean(in_bandwidth, axis=0)
                new_points.append(new_point)

            new_points = np.array(new_points)
            # Degisim miktari
            shift = np.linalg.norm(new_points - points, axis=1)
            if np.all(shift < self.tol):
                break

            points = new_points

        # Kume merkezleri belirlenir
        unique_points = np.unique(np.round(points, decimals=5), axis=0)
        self.cluster_centers_ = unique_points

        # Etiketler
        self.labels_ = np.array([np.argmin([np.sqrt(np.sum((x - center) ** 2)) for center in self.cluster_centers_]) for x in X])

    def predict(self, X):
        labels = np.array([np.argmin([np.sqrt(np.sum((x - center) ** 2)) for center in self.cluster_centers_]) for x in X])
        return labels

# Eğitim

In [20]:
ms = MeanShift(bandwidth=2.0)
ms.fit(X)

# Sonuçlar

In [23]:
cluster_centers = ms.predict(X)

In [24]:
print(f"Homogeneity: {metrics.homogeneity_score(y, cluster_centers):.3f}")
print(f"Completeness: {metrics.completeness_score(y, cluster_centers):.3f}")
print(f"V-measure: {metrics.v_measure_score(y, cluster_centers):.3f}")
print(f"Adjusted Rand Index: {metrics.adjusted_rand_score(y, cluster_centers):.3f}")
print(f"Adjusted Mutual Information: {metrics.adjusted_mutual_info_score(y, cluster_centers):.3f}")
print(f"Silhouette Coefficient: {metrics.silhouette_score(X, cluster_centers):.3f}")

Homogeneity: 1.000
Completeness: 0.557
V-measure: 0.715
Adjusted Rand Index: 0.498
Adjusted Mutual Information: 0.700
Silhouette Coefficient: 0.265
